In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
os.system("snscrape --jsonl --max-results 100 twitter-hashtag 'nctdisband' > tag1.json")

0

In [5]:
df = pd.read_json("tag1.json", lines = True)
df.head(2)

_type  \
0  snscrape.modules.twitter.Tweet   
1  snscrape.modules.twitter.Tweet   

                                                 url  \
0  https://twitter.com/karla59667320/status/13982...   
1  https://twitter.com/macy_yoo/status/1398216259...   

                       date  \
0 2021-05-28 10:17:47+00:00   
1 2021-05-28 09:55:08+00:00   

                                             content  \
0  @MapiArrojo no pongas el hashtag que apoyas a ...   
1  @mangooongo THAT fandom keep on dragging NCT, ...   

                                     renderedContent                   id  \
0  @MapiArrojo no pongas el hashtag que apoyas a ...  1398221959427616768   
1  @mangooongo THAT fandom keep on dragging NCT, ...  1398216259875995651   

                                                user  replyCount  \
0  {'_type': 'snscrape.modules.twitter.User', 'us...           0   
1  {'_type': 'snscrape.modules.twitter.User', 'us...           0   

   retweetCount  likeCount  ...  media  retweetedTweet quotedTweet  \
0             0          0  ...   None             NaN        None   
1             0          0  ...   None             NaN        None   

  inReplyToTweetId                                      inReplyToUser  \
0     1.397770e+18  {'_type': 'snscrape.modules.twitter.User', 'us...   
1     1.397861e+18  {'_type': 'snscrape.modules.twitter.User', 'us...   

                                      mentionedUsers coordinates place  \
0  [{'_type': 'snscrape.modules.twitter.User', 'u...         NaN   NaN   
1  [{'_type': 'snscrape.modules.twitter.User', 'u...         NaN   NaN   

                   hashtags  cashtags  
0  [nctdiamond, btsdiamond]       NaN  
1                      None       NaN  

[2 rows x 28 columns]

In [6]:
from pandas.io.json import json_normalize
import warnings

warnings.filterwarnings("ignore")

In [5]:
print(df.shape)

(100, 28)


In [7]:
#normalize user object
users = json_normalize(df['user'])
users.drop(['description', 'linkTcourl'], axis=1, inplace=True)
users.rename(columns={'id':'userId', 'url':'profileUrl'}, inplace=True)
users.head(2)

_type       username               userId  \
0  snscrape.modules.twitter.User  karla59667320  1398023433443811328   
1  snscrape.modules.twitter.User       macy_yoo  1316653278097072128   

  displayname               rawDescription descriptionUrls  verified  \
0       karla  nctzen/wayzeni\nTE AMO NCT😭            None     False   
1   Mariyeahh                trying hard 😶            None     False   

                     created  followersCount  friendsCount  statusesCount  \
0  2021-05-27T21:09:08+00:00               1            15             97   
1  2020-10-15T08:13:17+00:00               6            26             59   

   favouritesCount  listedCount  mediaCount                        location  \
0              103            0           1                                   
1             2931            0           5  Bicol Region, Republic of the    

   protected linkUrl                                    profileImageUrl  \
0      False    None  https://pbs.twimg.com/profile_images/139802360...   
1      False    None  https://pbs.twimg.com/profile_images/138009250...   

                                    profileBannerUrl  \
0                                               None   
1  https://pbs.twimg.com/profile_banners/13166532...   

                          profileUrl  
0  https://twitter.com/karla59667320  
1       https://twitter.com/macy_yoo

In [8]:
#drop duplicates
users = pd.DataFrame(users)
users.drop_duplicates(subset=['userId'], inplace=True)
users.head(2)
print(users.shape)

(53, 20)


In [9]:
#add userid column
user_id = []
for user in df['user']:
    uid = user['id']
    user_id.append(uid)
df['userId'] = user_id

#clean
cols = ['date', 'renderedContent', 'id', 'userId', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'media', 'retweetedTweet', 'quotedTweet', 'mentionedUsers', 'lang']
tweets = df[cols]
tweets.rename(columns={'id':'tweetId'}, inplace=True)
tweets.head(5)

date  \
0 2021-05-28 10:17:47+00:00   
1 2021-05-28 09:55:08+00:00   
2 2021-05-28 09:50:23+00:00   
3 2021-05-28 09:50:00+00:00   
4 2021-05-28 09:49:35+00:00   

                                     renderedContent              tweetId  \
0  @MapiArrojo no pongas el hashtag que apoyas a ...  1398221959427616768   
1  @mangooongo THAT fandom keep on dragging NCT, ...  1398216259875995651   
2  #NCTDREAM #NCT2020 nct dream hot sauce comebac...  1398215065010987011   
3  #NCTDREAM #NCT2020 nct dream hot sauce comebac...  1398214968365903872   
4  #NCTDREAM #NCT2020 nct dream hot sauce comebac...  1398214862786813952   

                userId  replyCount  retweetCount  likeCount  quoteCount  \
0  1398023433443811328           0             0          0           0   
1  1316653278097072128           0             0          0           0   
2  1334459256947449856           0             0          0           0   
3  1334459256947449856           0             0          0           0   
4  1334459256947449856           0             0          0           0   

                                               media  retweetedTweet  \
0                                               None             NaN   
1                                               None             NaN   
2  [{'_type': 'snscrape.modules.twitter.Video', '...             NaN   
3  [{'_type': 'snscrape.modules.twitter.Video', '...             NaN   
4  [{'_type': 'snscrape.modules.twitter.Video', '...             NaN   

  quotedTweet                                     mentionedUsers lang  
0        None  [{'_type': 'snscrape.modules.twitter.User', 'u...   es  
1        None  [{'_type': 'snscrape.modules.twitter.User', 'u...   en  
2        None                                               None   en  
3        None                                               None   en  
4        None                                               None   en

In [11]:
#convert to dataframe, drop duplicates, keep only english tweets
tweets = pd.DataFrame(tweets)
tweets.drop_duplicates(subset=['tweetId'], inplace=True)
print(tweets.shape)

(100, 13)


In [14]:
#work required
tweets = tweets['lang'='en']

SyntaxError: invalid syntax (<ipython-input-14-860ae5f8c779>, line 1)

In [12]:
tweets.to_csv('tag1.csv')